In [1]:
from collections.abc import Iterable
import numpy as np
import pandas as pd
from scipy import integrate, optimize, stats, signal
import matplotlib.pyplot as plt

 - https://www.wolframalpha.com/input?i=diff%5Br*C%5Ep%281-%28C%2FK%29%5Ea%29%2C+r%5D
 - https://www.wolframalpha.com/input?i=diff%5Br*C%5Ep%281-%28C%2FK%29%5Ea%29%2C+p%5D
 - https://www.wolframalpha.com/input?i=diff%5Br*C%5Ep%281-%28C%2FK%29%5Ea%29%2C+K%5D
 - https://www.wolframalpha.com/input?i=diff%5Br*C%5Ep%281-%28C%2FK%29%5Ea%29%2C+a%5D
 - https://www.wolframalpha.com/input?i=C%27%28t%29+%3D+r*C%5Ep%281-%28C%28t%29%2FK%29%5Ea%29

In [2]:
class GRMOld:
    
    def __init__(self, scale=None):
        self.scale = np.array(scale or [1e5, 1, 1, 1e5, 1])
    
    def ode(self, t, C, r, p, K, a):
        return r*(np.power(C, p))*(1 - np.power((C/K), a))

    def objective(self, t, C0, r, p, K, a, rtol=1e-9):
        #print(C, r, p, K, a)
        return integrate.solve_ivp(
            self.ode, (t[0], t[-1]), [C0], t_eval=t, args=(r, p, K, a), rtol=rtol
        ).y[0]
    
    def scaler(self, method):
        def wrapped(*args, **kwargs):
            return method(*args, **kwargs)/self.scale[0]
        return wrapped
    
    def solve(self, t, Ct, sigma=None, p0=None, bounds=None, gtol=1e-9, max_nfev=25000):
            
        # Initializing:
        if p0 is None:
            p0 = (1e4, 5e-1, 5e-1, 1e6, 5e-1)
            p0 = (17629.907849, 2.071898, 0.763188, 1.491130e+06, 2.133465)
        
        if bounds is None:
            bounds = ((1e0, 1e-1, 1e-3, 1e0, 1e-2), (1e6, 5e0, 1e0, 1e10, 1e1))
       
        # Scaling:
        p0 /= self.scale
        bounds = tuple([bound/self.scale for bound in bounds])
        
        # Solving:
        popt, pcov = optimize.curve_fit(
            self.scaler(self.objective),
            np.array(t), np.array(Ct)/self.scale[0], sigma=sigma,
            p0=p0, bounds=bounds, method="trf",
            gtol=gtol, max_nfev=max_nfev
        )
        
        # Unscaling:
        popt *= self.scale
        pcov *= np.outer(solver.scale, solver.scale)
        
        return popt, pcov

In [3]:
class GRM:
    
    def __init__(self, scale=None):
        self.scale = np.array(scale or [1, 1, 1, 1, 1])
    
    def ode(self, t, u, r, p, K, a, k=1e5):
        # C = k*u  => dC = k*du
        return (r/k)*(np.power(u*k, p))*(1 - np.power((u*k/K), a))
    
    def objective(self, t, C0, r, p, K, a, k=1e5, rtol=1e-9):
        #print(C, r, p, K, a)
        return integrate.solve_ivp(
            self.ode, (t[0], t[-1]), [C0], t_eval=t, args=(r, p, K, a), rtol=rtol
        ).y[0]*k
    
    def scaler(self, method):
        def wrapped(t, C, r, p, K, a, **kwargs):
            return method(t, C, r, p, K, a, **kwargs)
        return wrapped
    
    def solve(self, t, Ct, sigma=None, p0=None, bounds=None, gtol=1e-9, max_nfev=25000):
            
        # Initializing:
        if p0 is None:
            p0 = (1e4, 8e-1, 8e-1, 1e6, 2e0)
            #p0 = (17629.907849, 2.071898, 0.763188, 1.491130e+06, 2.133465)
        
        if bounds is None:
            bounds = ((1e0, 1e-1, 1e-3, 1e0, 1e-2), (1e6, 5e0, 1e0, 1e10, 1e1))
       
        # Scaling:
        p0 /= self.scale
        bounds = tuple([bound/self.scale for bound in bounds])
        
        # Solving:
        popt, pcov = optimize.curve_fit(
            self.objective,
            np.array(t), np.array(Ct)/1e5, sigma=sigma,
            p0=p0, bounds=bounds, method="trf",
            gtol=gtol, max_nfev=max_nfev
        )
        
        # Unscaling:
        popt *= self.scale
        pcov *= np.outer(solver.scale, solver.scale)
        
        return popt, pcov

In [4]:
data = pd.read_csv("peak.csv")

In [5]:
solver = GRM()

In [ ]:
p, cov = solver.solve(data["index_origin"], data["cs_savgol_origin"])
p, cov

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in power
  


In [ ]:
#p[1] = 2.3

In [ ]:
t = data["index_origin"].values
C = solver.objective(t, *p)#*solver.scale[0]

In [ ]:
plt.plot(t, C)

In [ ]:
axe = data.set_index("index_origin")[["cumsum_origin", "cs_savgol_origin"]].plot()
axe.plot(t, C, label="x")
axe.legend()

In [ ]:
axe = data.set_index("index_origin")[["cumsum_origin", "cs_savgol_origin"]].diff().plot()
axe.plot(t[:-1], np.diff(C))